In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
print(train.head())
train.columns
set(list(train["MSSubClass"][:]))

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD   

{20, 30, 40, 45, 50, 60, 70, 75, 80, 85, 90, 120, 160, 180, 190}

In [3]:
def data_prep_dict(dataFrame):
    columns = dataFrame.columns
    getUniqVals = lambda column : set(list(column))
    dataFrameDict = {}
    for column in columns:
        columnUniqVals = getUniqVals(dataFrame.loc[:, column])
        auxDict = {}
        for (idx, val) in enumerate(columnUniqVals):
            auxDict[str(val)] = idx
        dataFrameDict[column] = auxDict
        
    return dataFrameDict

print(data_prep_dict(train.head(15)))

{'Id': {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9, '11': 10, '12': 11, '13': 12, '14': 13, '15': 14}, 'MSSubClass': {'70': 0, '50': 1, '20': 2, '60': 3, '190': 4}, 'MSZoning': {'RL': 0, 'RM': 1}, 'LotFrontage': {'65.0': 0, 'nan': 7, '68.0': 2, '70.0': 4, '75.0': 5, '80.0': 6, '50.0': 8, '51.0': 9, '84.0': 10, '85.0': 11, '91.0': 12, '60.0': 13}, 'LotArea': {'9600': 0, '11200': 1, '8450': 2, '14115': 3, '10084': 4, '6120': 5, '12968': 6, '10920': 7, '9550': 8, '10382': 9, '11250': 10, '14260': 11, '11924': 12, '10652': 13, '7420': 14}, 'Street': {'Pave': 0}, 'Alley': {'nan': 0}, 'LotShape': {'Reg': 0, 'IR1': 1, 'IR2': 2}, 'LandContour': {'Lvl': 0}, 'Utilities': {'AllPub': 0}, 'LotConfig': {'FR2': 0, 'Inside': 1, 'Corner': 2}, 'LandSlope': {'Gtl': 0}, 'Neighborhood': {'NoRidge': 0, 'NAmes': 1, 'Sawyer': 2, 'Somerst': 3, 'Mitchel': 4, 'OldTown': 5, 'Veenker': 6, 'Crawfor': 7, 'BrkSide': 8, 'NridgHt': 9, 'CollgCr': 10, 'NWAmes': 11}, 'Condition1': {'Ar